## AES128 performance demo

In [ ]:
import numpy as np
from matplotlib import image
from matplotlib import pyplot
import matplotlib.pyplot as plt
np.set_printoptions(formatter={'int':hex})
from Crypto.Cipher import AES
import binascii
from siha import *
import pad
inputFile = 'data/encryptedSignal.bin'
outputHWFile = 'data/origSignalHW.bin'
outputSWFile = 'data/origSignalSW.bin'

# Initialization

In [ ]:
BUFFSIZE = 0x4000000
databuffOrg = np.fromfile(inputFile, dtype=np.uint8)[:BUFFSIZE]
databuff, padcount = pad.addpad(databuffOrg)
databuff = np.frombuffer(databuff.tobytes(), dtype=np.uint32)
size = databuff.shape[0]

encKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000001, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32)

decKeybuff = np.array([
    0x0c0d0e0f, 0x08090a0b, 0x04050607, 0x00010203,
    0x00000000, 0x00000000, 0x00000000, 0x00000000], dtype=np.uint32)

tid_conf = 1
tid_run = 0
offset_key = 0
offset_input = 64
offset_result = 64
size_config = len(encKeybuff)

# AES128 running on PL

In [ ]:
accel = AcapdAccel("AES", 0)
accel.loadData(decKeybuff, offset_key)
accel.loadData(databuff, offset_input)
accel.startAccel()
accel.config(offset_key, size_config, tid_conf)
accel.run(offset_input, size, tid_run, offset_result)
outbuff = accel.retrieve(offset_result, size)
outbuff = np.frombuffer(outbuff.tobytes(), dtype=np.uint8)
if padcount != 0:
    outbuff = outbuff[:-padcount]
accel.close()
outbuff.tofile(outputHWFile)

print("AES accel Time taken  : %f sec" % (accel.getTimeTaken()))
print("AES In Throughput  : %f gbps" % (accel.getInThroughput()))
print("AES Out Throughput  : %f gbps" % (accel.getOutThroughput()))

# AES128 running on PS

In [ ]:
with open(inputFile, "rb") as f:
    softData = f.read()

key = '0f0e0d0c0b0a09080706050403020100'
key = binascii.unhexlify(key)
encryptor = AES.new(key, AES.MODE_ECB)
start = time.time()
out = encryptor.decrypt(softData)
stop = time.time()
inThroughput = 8*len(softData)/(stop - start)/1000000000
outThroughput = 8*len(softData)/(stop - start)/1000000000
file = open(outputSWFile, "wb")
file.write(out)
file.close()

print("AES accel Time taken  : %f sec" % (stop-start))
print("AES In Throughput  : %f gbps" % (inThroughput))
print("AES Out Throughput  : %f gbps" % (outThroughput))